In [1]:
import json
import os
from pathlib import Path
import asyncio

import duckdb
import requests
import nest_asyncio


In [2]:
#Asyncio versiopn, not working due to problems with aiohttp

nest_asyncio.apply()

async def fetch_movie_details(movie_name: str, api_key: str) -> dict:
    return await requests.get(f"https://www.omdbapi.com/?t={movie_name}&apikey=6a6418c2").json()
    

async def read_movies_details_async(movies, api_key):
    tasks = [asyncio.ensure_future(fetch_movie_details(movie, api_key)) for movie in movies]
    return await asyncio.gather(*tasks)

def read_movies_details(movies, api_key):
    return asyncio.run(read_movies_details_async(movies, api_key))


In [3]:
def fetch_movie_details(movie_name: str, api_key: str) -> dict:
    url = f"https://www.omdbapi.com/?t={movie_name}&apikey=6a6418c2"
    response = requests.get(url).json()
    response["request_url"]=url.rsplit("&apikey=")[0]
    return response

def read_movies_details(movies, api_key: str) ->list[dict]:
    output = []
    for m in movies:
        response = fetch_movie_details(m, api_key)
        if response.get("Response")=="True":
            output.append(response)
    return output

In [ ]:
def save_json_file(path: str, json_content: dict) -> None:
    output_file = Path(path)
    output_file.parent.mkdir(exist_ok=True, parents=True)
    with output_file.open('w') as f:
        json.dump(json_content, f, indent=4)


def filter_movies(movies: list[str], movies_existing: list[str]) -> list[str]:
    return list(set(movies) - set(movies_existing))



movies = [m[0] for m in duckdb.sql("SELECT DISTINCT title from 'data/landing_zone/revenues_per_day.csv'").fetchall()]
movies_details_file = Path('data/landing_zone/movies_details.json')
if movies_details_file.is_file():
    with open('data/landing_zone/movies_details.json', "r") as f:
        existing_movies = json.loads(f.read()) or []
else: 
    existing_movies = []
movies_to_read = filter_movies(movies, [m['Title'] for m in existing_movies])
data = read_movies_details(movies_to_read, "6a6418c2")

save_json_file('data/landing_zone/movies_details.json', existing_movies+data)